#Obtain the dataset from the GitHub repository

In [ ]:
!git clone https://github.com/iosifache/DikeDataset.git

Cloning into 'DikeDataset'...
remote: Enumerating objects: 11973, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (23/23), done.
remote: Total 11973 (delta 6), reused 23 (delta 4), pack-reused 11946
Receiving objects: 100% (11973/11973), 2.38 GiB | 17.42 MiB/s, done.
Resolving deltas: 100% (2739/2739), done.
Checking out files: 100% (11935/11935), done.


#Import the necessary modules and libraries

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Activation, MaxPooling2D, Flatten, Dense, Dropout
import numpy as np
import pickle
import random
import os

#Reading integers from the bytes in each file

In [ ]:
def read_ints_from_file(file_name, file_size):
  image_numpy_arr = np.zeros((256, 256), dtype=int)
  max_file_size = 256*256
  curr_file = open(file_name, "rb")
  bytes_read = 0
  row_num = 0
  col_num = 0
  try:
    byte = curr_file.read(1)
    curr_int = int.from_bytes(byte, "little")
    bytes_read += 1
    image_numpy_arr[row_num][col_num] = curr_int
    while bytes_read <= file_size and bytes_read < max_file_size and row_num < 255:
      byte = curr_file.read(1)
      curr_int = int.from_bytes(byte, "little")
      if col_num < 255:
        col_num += 1
        image_numpy_arr[row_num][col_num] = curr_int
      else:
        row_num += 1
        col_num = 0
        image_numpy_arr[row_num][col_num] = curr_int
  finally:
    curr_file.close()
    return image_numpy_arr

# Generating the training data

In [ ]:
# will only use 1,000 of each file to save time and space
def generate_training_data():
  training_data = []
  # 0 for benign files, 1 for malware files
  classifications = ["benign", "malware"]
  sample_files_dir = "DikeDataset/files"
  for classification in classifications:
    path = os.path.join(sample_files_dir, classification)
    label_num = classifications.index(classification)
    for file in os.listdir(path)[:1000]:
      file_name = os.path.join(path, file)
      file_size = os.path.getsize(file_name)
      img_array = read_ints_from_file(file_name, file_size)
      training_data.append([img_array, label_num])
  return training_data

# shuffle the training data so the neural network can effectively learn
training_data = generate_training_data()
random.shuffle(training_data)

# Extracting the features and labels from training data

In [ ]:
X = []
y = []
for feature, label in training_data:
  X.append(feature)
  y.append(label)

# reshape the data before entering the neural network
X = np.array(X).reshape(-1, 256, 256, 1)

# Use the Pickle library to store and load your data

In [ ]:
pickle_file = open("X.pickle", "wb")
pickle.dump(X, pickle_file)
pickle_file.close()

pickle_file = open("y.pickle", "wb")
pickle.dump(y, pickle_file)
pickle_file.close()

In [ ]:
pickle_read = open("X.pickle", "rb")
X = pickle.load(pickle_read)

pickle_read = open("y.pickle", "rb")
y = pickle.load(pickle_read)

X = X/255
y = np.array(y)

# Training the neural network to recognize malware

In [ ]:
model = Sequential()
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(64))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(X, y, batch_size=20, epochs=10, validation_split=0.1)

Epoch 1/10
90/90 [==============================] - 6s 57ms/step - loss: 1.0840 - accuracy: 0.8167 - val_loss: 0.1896 - val_accuracy: 0.9600
Epoch 2/10
90/90 [==============================] - 5s 54ms/step - loss: 0.1587 - accuracy: 0.9467 - val_loss: 0.1657 - val_accuracy: 0.9300
Epoch 3/10
90/90 [==============================] - 5s 55ms/step - loss: 0.0939 - accuracy: 0.9711 - val_loss: 0.1085 - val_accuracy: 0.9650
Epoch 4/10
90/90 [==============================] - 5s 54ms/step - loss: 0.0447 - accuracy: 0.9867 - val_loss: 0.1067 - val_accuracy: 0.9700
Epoch 5/10
90/90 [==============================] - 5s 56ms/step - loss: 0.0213 - accuracy: 0.9961 - val_loss: 0.1187 - val_accuracy: 0.9700
Epoch 6/10
90/90 [==============================] - 5s 55ms/step - loss: 0.0087 - accuracy: 0.9983 - val_loss: 0.1282 - val_accuracy: 0.9600
Epoch 7/10
90/90 [==============================] - 5s 54ms/step - loss: 0.0105 - accuracy: 0.9983 - val_loss: 0.1423 - val_accuracy: 0.9700
Epoch 8/10
90